# Mangelfulle spørringer mot NVDB api V3? 

her sammenligner vi resultatet av spørringen `/vegobjekter/904?kommune=5001&veg(system)referanse=K` mot NVDB api V2 og V3.

In [10]:
import json
import pandas as pd
import sjekkmengdeuttak
import requests
pd.set_option('display.max_rows', 500)

In [2]:
with open( 'trondheim_905/mengdeuttak_v2.json') as f:
    v2data = pd.DataFrame( json.load(f) )

with open( 'trondheim_905/mengdeuttak_v3.json') as f:
    v3data = pd.DataFrame( json.load(f) )


# Savner vi noen vegobjekter i V3-spørringen? 

In [3]:
v2data[ ~v2data['vegobjektid'].isin( v3data['vegobjektid'].tolist()  )]

,apiversjon,vegobjektid,href,versjon,startdato,geometri,geometritype,wkt_punkt,kommuner,vegkat,vref,stedfesting_antall,stedfesting_utbredelse,stedfesting_unikeid


Nope, tomt datasett = V3-spørringen får med alle objektene. 

# Får vi for mange vegobjekter i V3-spørringen? 

In [9]:
v3formye = v3data[ ~v3data['vegobjektid'].isin( v2data['vegobjektid'].tolist() )]
print( 'vi har', len(v3formye), 'objekter i V3-spørringen som ikke finnes i V2')


vi har 159 objekter i V3-spørringen som ikke finnes i V2


### La oss se om noen av de overflødige har gyldig geometri? 

In [5]:
v3formye[ v3formye['geometri']] # geometri=True hvis det er en WKT-streng med gyldige data

,apiversjon,vegobjektid,href,versjon,startdato,geometri,geometritype,wkt_punkt,kommuner,vegkat,vref,stedfesting_antall,stedfesting_utbredelse,stedfesting_unikeid


### Niks. La oss hente dem en og en fra V3 og titte på dem

In [6]:
sjekkdata = []
headers =   { 'accept' : 'application/vnd.vegvesen.nvdb-v3-rev1+json', 
                            'X-Client' : 'nvdbapi.py',
                            'X-Kontaktperson' : 'jan.kristian.jensen@vegvesen.no'}


for ix, row in v3formye.iterrows():
    r = requests.get( row['href'], headers=headers)
    sjekkdata.append( sjekkmengdeuttak.plukkutdata( r.json()))
    
    

In [11]:
sjekket = pd.DataFrame( sjekkdata)
sjekket[['geometri', 'vegkat', 'vref' ]]

,geometri,vegkat,vref
0,True,EP,"EV39 S1D30 m75-287,PV7388 S4D1 m0-9"
1,True,P,PV4075 S2D1 m0-353
2,True,P,PV95983 S1D1 m0-858
3,True,P,PV8475 S7D1 m0-29
4,True,P,PV8475 S8D1 m0-274
5,True,P,PV8475 S6D1 m0-319
6,True,P,PV8165 S8D1 m0-85
7,True,P,"PV8117 S4D1 m0-27,PV8117 S4D1 m27-256"
8,True,P,"PV2380 S1D1 m67-91,PV2380 S1D1 m9-67,PV2380 S2..."
9,True,P,PV8030 S3D1 m0-133
